In [99]:
# Import Libraries and Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from fuzzywuzzy import process
import pandas as pd
import requests
import time

# World Cities DB Cleaning

In [100]:
# Read csv and create DF
worldcities_df = pd.read_csv("./RESOURCES/worldcities.csv")
worldcities_df.drop(columns=worldcities_df.columns[0], inplace=True)

worldcities_df.head()

,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000.0,1392685764
1,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
2,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629


In [101]:
# Adjust the column name as necessary based on your dataset
filtered_df = worldcities_df[worldcities_df['country'] == 'Mexico'].copy()

# Display the first few rows of the filtered data
filtered_df.head()

,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
9,Mexico City,19.4333,-99.1333,Mexico,MX,MEX,Ciudad de México,primary,21804000.0,1484247881
109,Guadalajara,20.6767,-103.3475,Mexico,MX,MEX,Jalisco,admin,5525000.0,1484950208
115,Monterrey,25.6667,-100.3000,Mexico,MX,MEX,Nuevo León,admin,5341171.0,1484559591
416,Tijuana,32.5250,-117.0333,Mexico,MX,MEX,Baja California,minor,2002000.0,1484708778
427,Ecatepec,19.6097,-99.0600,Mexico,MX,MEX,México,minor,1929926.0,1484003694


In [102]:
# Check how many rows there are in the filtered worldcities_df
num_rows = filtered_df.shape[0]
print(f"Number of rows in the filtered DataFrame: {num_rows}")

Number of rows in the filtered DataFrame: 983


In [103]:
filtered_df.columns

Index(['city_ascii', 'lat', 'lng', 'country', 'iso2', 'iso3', 'admin_name',
       'capital', 'population', 'id'],
      dtype='object')

In [104]:
filtered_df.drop(columns=['country', 'iso2', 'iso3', 'capital', 'population'], inplace=True)
filtered_df.columns

Index(['city_ascii', 'lat', 'lng', 'admin_name', 'id'], dtype='object')

In [105]:
filtered_df.head()

,city_ascii,lat,lng,admin_name,id
9,Mexico City,19.4333,-99.1333,Ciudad de México,1484247881
109,Guadalajara,20.6767,-103.3475,Jalisco,1484950208
115,Monterrey,25.6667,-100.3000,Nuevo León,1484559591
416,Tijuana,32.5250,-117.0333,Baja California,1484708778
427,Ecatepec,19.6097,-99.0600,México,1484003694


In [106]:
# Rename the columns
filtered_df.rename(columns={
    'city_ascii': 'City',
    'admin_name': 'State',
    'lat': 'Lat',
    'lng': 'Lng',
    'id': 'ID'
}, inplace=True)

# Reorder the columns
filtered_df = filtered_df[['City', 'State', 'Lat', 'Lng', 'ID']]

filtered_df.head()

,City,State,Lat,Lng,ID
9,Mexico City,Ciudad de México,19.4333,-99.1333,1484247881
109,Guadalajara,Jalisco,20.6767,-103.3475,1484950208
115,Monterrey,Nuevo León,25.6667,-100.3000,1484559591
416,Tijuana,Baja California,32.5250,-117.0333,1484708778
427,Ecatepec,México,19.6097,-99.0600,1484003694


In [107]:
# Save the modified DataFrame to a new CSV file
filtered_df.to_csv('./RESOURCES/mexico_cities.csv', index=False)

# NUFORC Mexico DB Cleaning

In [108]:
# Read csv and create DF
UFO_df = pd.read_csv("./RESOURCES/nuforc_mexico_reports_detailed.csv")
UFO_df.drop(columns=UFO_df.columns[0], inplace=True)

UFO_df.head()

,Occurred,City,State,Country,Shape,Summary,Reported,Media,Explanation,Detail URL,Detailed Description
0,05/08/1998 22:30,Morelos (rural area) (Mexico),NaN,Mexico,Light,We where on the road when we saw 2 long lights...,07/27/1998,NaN,NaN,https://nuforc.org/sighting/?id=4301,NaN
1,03/15/1996 03:00,San Felipe (driving south to) (Mexico),NaN,Mexico,Diamond,While traveling to San Felipe for a fishing tr...,03/12/1998,NaN,NaN,https://nuforc.org/sighting/?id=3323,NaN
2,09/15/1998 21:00,"Wadley, San Luis Potosi, SLP (In the desert ne...",NaN,Mexico,Disk,"Craft was approaching, it accelerated very fas...",11/10/1998,NaN,NaN,https://nuforc.org/sighting/?id=4800,NaN
3,09/15/1994 10:00,Mexicali B. C. Mexico; Sierra Cucapah; Cerro p...,NaN,Mexico,Oval,I seen one objet upper the Sierra cucapah (Cuc...,01/03/1998,NaN,NaN,https://nuforc.org/sighting/?id=3007,NaN
4,12/05/1998 19:00,Valle de Bravo (Mexico),NaN,Mexico,Light,White bright light like a headlight with a clo...,12/06/1998,NaN,NaN,https://nuforc.org/sighting/?id=5016,NaN


In [109]:
UFO_df.columns

Index(['        Occurred', '        City', '        State', '        Country',
       '        Shape', '        Summary', '        Reported', '        Media',
       '        Explanation', 'Detail URL', 'Detailed Description'],
      dtype='object')

In [110]:
# Delete spaces on headers (required for the next step, since there are spaces before/after titles)
# For example [' Explanation '] to ['Explanation']
UFO_df.columns = [col.strip() for col in UFO_df.columns]

# Drop columns that don't give relevant information
UFO_df.drop(columns=['Explanation', 'Reported', 'Detailed Description', 'Media'], inplace=True)

# Display the updated DataFrame
print("\nUpdated DataFrame:")
print(UFO_df.columns)


Updated DataFrame:
Index(['Occurred', 'City', 'State', 'Country', 'Shape', 'Summary',
       'Detail URL'],
      dtype='object')


In [111]:
len(UFO_df)

318

In [112]:
# Create a new column with the information that is in parenthesis in City
UFO_df['Notas'] = UFO_df['City'].str.extract(r'\((.*?)\)', expand=False)

# Delete the information in parenthesis in the column City
UFO_df['City'] = UFO_df['City'].str.replace(r'\((.*?)\)', '', regex=True).str.strip()

# Remove any accidental duplicates created
UFO_df = UFO_df.drop_duplicates()

UFO_df.head()

,Occurred,City,State,Country,Shape,Summary,Detail URL,Notas
0,05/08/1998 22:30,Morelos,NaN,Mexico,Light,We where on the road when we saw 2 long lights...,https://nuforc.org/sighting/?id=4301,rural area
1,03/15/1996 03:00,San Felipe,NaN,Mexico,Diamond,While traveling to San Felipe for a fishing tr...,https://nuforc.org/sighting/?id=3323,driving south to
2,09/15/1998 21:00,"Wadley, San Luis Potosi, SLP",NaN,Mexico,Disk,"Craft was approaching, it accelerated very fas...",https://nuforc.org/sighting/?id=4800,In the desert near
3,09/15/1994 10:00,Mexicali B. C. Mexico; Sierra Cucapah; Cerro p...,NaN,Mexico,Oval,I seen one objet upper the Sierra cucapah (Cuc...,https://nuforc.org/sighting/?id=3007,NaN
4,12/05/1998 19:00,Valle de Bravo,NaN,Mexico,Light,White bright light like a headlight with a clo...,https://nuforc.org/sighting/?id=5016,Mexico


In [113]:
len(UFO_df)

318

# Checking for typos and Assigning State

In [114]:
# Function to find the best match for each city name
def find_best_match(city, choices):
    if pd.isna(city):
        return city
    match, score = process.extractOne(str(city), choices)
    return match if score > 80 else city  # Return the original city if no good match is found

# Ensure all values in the 'City' column are treated as strings
UFO_df['City'] = UFO_df['City'].astype(str)

# List of known correct city names
known_cities = filtered_df['City'].tolist()

# Correct the city names in UFO_df
UFO_df['Corrected_City'] = UFO_df['City'].apply(lambda x: find_best_match(x, known_cities))

# Remove duplicates in filtered_df by keeping only the first occurrence of each city
filtered_df_unique = filtered_df.drop_duplicates(subset='City', keep='first')

# Create a dictionary from filtered_df_unique for quick lookup
city_state_dict = filtered_df_unique.set_index('City')['State'].to_dict()

# Assign the correct state based on the corrected city
UFO_df['State_corrected'] = UFO_df['Corrected_City'].map(city_state_dict)

# Replace the information in the 'City' column with 'Corrected_City'
UFO_df['City'] = UFO_df['Corrected_City']

# Replace the information in the 'State' column with 'State_corrected'
UFO_df['State'] = UFO_df['State_corrected']

# Drop rows where the 'State' column is empty
UFO_df.dropna(subset=['State'], inplace=True)

# Drop the redundant 'Corrected_City' and 'State_corrected' columns
UFO_df.drop(columns=['Corrected_City', 'State_corrected', 'Notas'], inplace=True)

# Save the modified DataFrame to a new CSV file
UFO_df.to_csv('./RESOURCES/NUFORC_Mexico_clean.csv', index=False)

# Display the final DataFrame
UFO_df.head()

,Occurred,City,State,Country,Shape,Summary,Detail URL
0,05/08/1998 22:30,Tepatitlan de Morelos,Jalisco,Mexico,Light,We where on the road when we saw 2 long lights...,https://nuforc.org/sighting/?id=4301
1,03/15/1996 03:00,San Felipe,Guanajuato,Mexico,Diamond,While traveling to San Felipe for a fishing tr...,https://nuforc.org/sighting/?id=3323
2,09/15/1998 21:00,San Luis Potosi,San Luis Potosí,Mexico,Disk,"Craft was approaching, it accelerated very fas...",https://nuforc.org/sighting/?id=4800
3,09/15/1994 10:00,Mexicali,Baja California,Mexico,Oval,I seen one objet upper the Sierra cucapah (Cuc...,https://nuforc.org/sighting/?id=3007
4,12/05/1998 19:00,Valle de Bravo,México,Mexico,Light,White bright light like a headlight with a clo...,https://nuforc.org/sighting/?id=5016


In [115]:
len(UFO_df)

284

## This is what was before

In [ ]:
# Check the cities that are contained the most to update the state

UFO_df['City'].value_counts()

In [ ]:
repeated_cities = {
    "Cancun": "Quintana Roo",
    "Puerto Vallarta": "Jalisco",
    "Mexico City": "Mexico City",
    "Tijuana": "Baja California"
}

# Assign state based on city
def assing_city(row):
    for city, state in repeated_cities.items():
        if pd.notna(row['City']) and city.lower() in row['City'].lower():
            return state
    return row['State']

# Apply the function
UFO_df['State'] = UFO_df.apply(assing_city, axis=1)


In [ ]:
# Set a list of the name of the Mexican States
state_names = ['Aguascalientes', 'Baja California', 'Baja California Sur', 'Campeche', 'Chiapas', 'Chihuahua', 'Coahuila', 'Colima', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Estado de Mexico', 'Michoacan', 'Morelos', 'Nayarit', 'Nuevo Leon', 'Oaxaca', 'Puebla', 'Queretaro', 'Quintana Roo', 'San Luis Potosi', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatan', 'Zacatecas','Mexico City']

#define a function that loops on the city name to find the state
def update_state(row):
    for state in state_names:
        if pd.notna(row['City']) and state.lower() in row['City'].lower():
            return state
    return row['State']

# Apply the function to the State column
UFO_df['State'] = UFO_df.apply(update_state, axis=1)

UFO_df

In [ ]:
# Check if the state is in the Notas column if the State is still not contained in the column

def update_withnote(row):
    if pd.isna(row['State']): 
        for state in state_names:
            if pd.notna(row['Notas']) and state.lower() in row['Notas'].lower():
                return state
    return row['State'] 

UFO_df['State'] = UFO_df.apply(update_withnote, axis=1)

UFO_df

SCRAPPING

In [ ]:
# Set up Splinter
browser = Browser('chrome')

In [ ]:
# Visit the NUFORC REPORTS UFO SIGHTINGS site
url = 'https://nuforc.org/subndx/?id=cMexico'
browser.visit(url)

In [ ]:
# Scrape the website
html = browser.html

# Create a BeautifulSoup object from the scraped HTML
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Initialize variables to store scraped data
all_data = []  # List to store table data
links = []     # List to store links
# Loop until the "Next" link is clicked 4 times
# The underscore means "iterate four times, and I don't care about the loop variable."
for _ in range(4):
    
    ''' EXTRACT DATA AND LINKS FROM THE TABLE INTO LISTS
     '#table_1 tr' is a CSS selector used to select all <tr> (table row) elements within
     the HTML table that has an id attribute equal to 'table_1'.'''
    
    table_rows = soup.select('#table_1 tr')[1:]  # # the [1:] means skip the first row (headers)
    for row in table_rows:
        # Extract text data from each <td> element in the row and removes leading and trailing whitespace characters from each extracted text
        row_data = [td.get_text(strip=True) for td in row.find_all('td')]
        all_data.append(row_data)  # Append row data to the list of all data
        # Extract links from <a> elements in the row
        row_links = [f"https://nuforc.org{a['href']}" for a in row.find_all('a', href=True)]
        links.extend(row_links)  # Extend the list of links with links from the current row
    # Find the "Next" link and click it
    next_link = browser.find_by_id('table_1_next')
    next_link.click()
    # Wait for the page to load
    time.sleep(2)  # Adjust the sleep time as needed
    # Update the HTML content after navigating to the next page
    html = browser.html
    # Create a new BeautifulSoup object from the updated HTML
    soup = BeautifulSoup(html, 'html.parser')
# Extract headers from the table
headers = [header.get_text(strip=True) for header in soup.select('#table_1 th')]
# Convert the list of lists into a pandas DataFrame
table_df = pd.DataFrame(all_data, columns=headers)  # Create DataFrame for table data
links_df = pd.DataFrame(links, columns=['Link'])    # Create DataFrame for links


In [ ]:
    # Display the table_df
table_df

In [ ]:
# Display the links_df
links_df.head()

In [ ]:
# Drop the first column in the table_df
table_df.drop(columns=table_df.columns[0], inplace=True)

# Merge both dataframes to include the links column (replacing the dropped one)
merged_df = pd.merge(table_df, links_df, left_index=True, right_index=True)
merged_df.head()

In [ ]:
base_url = "https://nuforc.org/subndx/?id=cMexico&page="
# Número de páginas a scraper
num_pages = 5  # Ajusta este valor según sea necesario
# Crear un DataFrame para almacenar los datos
df = pd.DataFrame()
# Iterar sobre las páginas
for page in range(1, num_pages + 1):
    # Construir la URL para la página actual
    url = base_url + str(page)
    # Enviar solicitud GET para la página actual
    response = requests.get(url)
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    # Encontrar la tabla que contiene los datos
    table = soup.find('table', {'class': 'table-striped'})
    # Extraer los encabezados de la tabla
    headers = [th.text.strip() for th in table.find('tr').find_all('th')]
    # Extraer los datos de la tabla
    data = []
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        data.append([col for col in cols])
    # Convertir los datos en un DataFrame de Pandas
    page_df = pd.DataFrame(data, columns=headers)
    # Agregar los datos de la página actual al DataFrame principal
    df = pd.concat([df, page_df], ignore_index=True)
# Mostrar el DataFrame completo
print(df.head())